In [52]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local') \
        .appName('Quiz1') \
        .config('spark.ui.port', '4050') \
        .getOrCreate()

sc = spark.sparkContext

문제 1.
* 5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?
* 'T'는 대문자
* 주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!
* 제한조건: 위 코드를 이용해 u.data와 u.item을 DataFrame으로 읽어들여 select및 join을 이용하여 계산하시오.
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.
* 힌트: filter를 수행 시 filter조건을 표현할 때 어떤 column의 부분문자열을 뽑기 위해 pyspark.sql.functions.substring를 사용할 수 있음. 예를 들어 uitem이란 data frame의 title이란 column의 첫 글자는 substring(uitem["title"], 0, 1)으로 지정할 수 있음.

In [53]:
from pyspark.sql.types import *

udataschema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate", LongType(), False),
    StructField("timestamp", LongType(), False)
])

udata = spark.read.format("csv").option("sep", "\t").schema(udataschema).load("../../data/input3/u.data")

uitemschema = StructType([
    StructField("mid", LongType(), False),
    StructField("title", StringType(), False)
])

uitem = spark.read.format("csv").option("sep", "|").schema(uitemschema).load("../../data/input3/u.item")

In [54]:
t_uitem = uitem.selectExpr('*', 'SUBSTR(title, 1, 1) as start').where('start="T"')
t_uitem.show(5)

+---+--------------------+-----+
|mid|               title|start|
+---+--------------------+-----+
|  1|    Toy Story (1995)|    T|
|  7|Twelve Monkeys (1...|    T|
| 23|  Taxi Driver (1976)|    T|
| 40|To Wong Foo, Than...|    T|
| 59|Three Colors: Red...|    T|
+---+--------------------+-----+
only showing top 5 rows



In [55]:
udata5 = udata.selectExpr('*').where('rate=5').distinct()
udata5.show(5)

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|  7|451|   5|891353892|
|112|289|   5|884992690|
|244|652|   5|880606533|
| 16|655|   5|877724066|
|321|483|   5|879439658|
+---+---+----+---------+
only showing top 5 rows



In [57]:
udata5.join(t_uitem, t_uitem['mid'] == udata5['mid'], 'inner').selectExpr('COUNT(DISTINCT(title))').show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                   75|
+---------------------+



문제 2.
* 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이는?
* 동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함
* 힌트: 어떤 column의 길이의 max 를 select할 때 max 함수와 length함수를 사용. 예를 들어 uitem["title"]의 길이의 max는 select(max(length(uitem["title"]))) 로 계산 할 수 있음
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

In [38]:
udata15 = udata.selectExpr('*').where('rate=5 OR rate=1').distinct()
udata15.show(5)

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|269|252|   1|891456350|
|  7|451|   5|891353892|
|112|289|   5|884992690|
|244|652|   5|880606533|
| 38|122|   1|892434801|
+---+---+----+---------+
only showing top 5 rows



In [41]:
udata15.join(uitem, uitem['mid'] == udata5['mid'], 'inner').selectExpr('MAX(LENGTH(title))').show()

+------------------+
|max(length(title))|
+------------------+
|                81|
+------------------+



In [58]:
sc.stop()